# Part 5 - LangGraph & Agents: Building Intelligent Workflows

Build complex workflows with states, nodes, and edges.

In [ ]:
!pip install -q langgraph langchain langchain-openai

In [ ]:
import os
import getpass

if not os.getenv('OPENAI_API_KEY'):
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OpenAI API key: ')

## Simple Graph Example

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    query: str
    result: str

def step1(state: State):
    print(f'Processing: {state["query"]}')
    return {'result': f'Processed: {state["query"]}'}

workflow = StateGraph(State)
workflow.add_node('process', step1)
workflow.add_edge('process', END)
workflow.set_entry_point('process')

app = workflow.compile()
result = app.invoke({'query': 'Hello World'})
print(result)

## Agent with Tools

In [ ]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

@tool
def search(query: str) -> str:
    '''Search for information.'''
    return f'Search results for: {query}'

model = ChatOpenAI(model='gpt-4')
agent = create_react_agent(model, [search])

response = agent.invoke({'messages': [('user', 'Search for LangGraph tutorials')]})
print(response['messages'][-1].content)

## Conditional Flow

In [ ]:
def router(state: State) -> str:
    if '?' in state['query']:
        return 'search'
    return 'answer'

workflow = StateGraph(State)
workflow.add_node('check', lambda s: s)
workflow.add_node('search', lambda s: {'result': 'searched'})
workflow.add_node('answer', lambda s: {'result': 'answered directly'})

workflow.add_conditional_edges('check', router, {'search': 'search', 'answer': 'answer'})
workflow.add_edge('search', END)
workflow.add_edge('answer', END)
workflow.set_entry_point('check')

app = workflow.compile()
result = app.invoke({'query': 'What is AI?'})
print(result)